In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
dataset = pd.read_csv('ratings.csv')
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
n_users = len(dataset.user_id.unique())
n_books = len(dataset.book_id.unique())

In [19]:
dataset.shape

(981756, 3)

In [20]:
n_users

53424

In [14]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense
from keras.models import Model
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [15]:
history = model.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)


Epoch 1/10
785404/785404 [==============================] - 45s 57us/step - loss: 13.55790s - loss: 13.
Epoch 2/10
785404/785404 [==============================] - 45s 58us/step - loss: 3.8673
Epoch 3/10
785404/785404 [==============================] - 45s 58us/step - loss: 1.5987
Epoch 4/10
785404/785404 [==============================] - 47s 59us/step - loss: 1.1232
Epoch 5/10
785404/785404 [==============================] - 46s 59us/step - loss: 0.9324
Epoch 6/10
785404/785404 [==============================] - 44s 56us/step - loss: 0.8340
Epoch 7/10
785404/785404 [==============================] - 43s 55us/step - loss: 0.7776
Epoch 8/10
785404/785404 [==============================] - 44s 56us/step - loss: 0.7428
Epoch 9/10
785404/785404 [==============================] - 44s 55us/step - loss: 0.7199
Epoch 10/10
785404/785404 [==============================] - 46s 59us/step - loss: 0.7036


In [16]:
#save the model
model.save('regression_model.h5')

In [17]:
# Extract embeddings
book_em = model.get_layer('Book-Embedding')
book_em_weights = book_em.get_weights()[0]


In [23]:
#from sklearn.manifold import TSNE
#tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
#tnse_results = tsne.fit_transform(book_em_weights)
#sns.scatterplot(x=tnse_results[:,0], y=tnse_results[:,1])

In [24]:
#from sklearn.decomposition import PCA
#import seaborn as sns
#pca = PCA(n_components=2)
#pca_result = pca.fit_transform(book_em_weights)
#sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1])

In [26]:
# Creating dataset for making recommendations for the first user
import numpy as np
book_data = np.array(list(set(dataset.book_id)))
user = np.array([1 for i in range(len(book_data))])
predictions = model.predict([user, book_data])
predictions = np.array([a[0] for a in predictions])
recommended_book_ids = (-predictions).argsort()[:5]
print(recommended_book_ids)
print(predictions[recommended_book_ids])

[7843 9841 8998 8925 8232]
[4.701331  4.4873524 4.422167  4.3669844 4.339404 ]


In [ ]:
#A few things you could add to get better results:

#Add fully-connected layers after the dot product
#Train for more epochs
#Scale the ratings column